In [1]:
import numpy as np
import pandas as pd

In [2]:
%%time
import cx_Oracle
import numpy as np
import pandas as pd

conn=cx_Oracle.connect('c##hr/hr@localhost:1521/xe')
cursor = conn.cursor()

Wall time: 552 ms


In [18]:
%%time
cursor.execute('SELECT * FROM halfyear')
x = cursor.fetchall()


dfhalf = pd.DataFrame(x, columns=['고객번호','반기','구매횟수','총구매액'])
dfhalf


Wall time: 240 ms


,고객번호,반기,구매횟수,총구매액
0,00001,14fw,41,376140
1,00001,14ss,1,21800
2,00001,15fw,10,184900
3,00001,15ss,12,208136
4,00002,14fw,7,46450
...,...,...,...,...
62742,19381,15fw,424,3143508
62743,19381,15ss,367,2471071
62744,19382,15fw,505,2567044
62745,19382,15ss,394,1651639


In [21]:
grp=dfhalf.groupby(['고객번호','반기'])
grp.describe()

구매횟수                                                총구매액  \
           count   mean std    min    25%    50%    75%    max count   
고객번호  반기                                                               
00001 14fw   1.0   41.0 NaN   41.0   41.0   41.0   41.0   41.0   1.0   
      14ss   1.0    1.0 NaN    1.0    1.0    1.0    1.0    1.0   1.0   
      15fw   1.0   10.0 NaN   10.0   10.0   10.0   10.0   10.0   1.0   
      15ss   1.0   12.0 NaN   12.0   12.0   12.0   12.0   12.0   1.0   
00002 14fw   1.0    7.0 NaN    7.0    7.0    7.0    7.0    7.0   1.0   
...          ...    ...  ..    ...    ...    ...    ...    ...   ...   
19381 15fw   1.0  424.0 NaN  424.0  424.0  424.0  424.0  424.0   1.0   
      15ss   1.0  367.0 NaN  367.0  367.0  367.0  367.0  367.0   1.0   
19382 15fw   1.0  505.0 NaN  505.0  505.0  505.0  505.0  505.0   1.0   
      15ss   1.0  394.0 NaN  394.0  394.0  394.0  394.0  394.0   1.0   
19383 15fw   1.0   16.0 NaN   16.0   16.0   16.0   16.0   16.0   1.0   

                                                                       \
                 mean std        min        25%        50%        75%   
고객번호  반기                                                                
00001 14fw   376140.0 NaN   376140.0   376140.0   376140.0   376140.0   
      14ss    21800.0 NaN    21800.0    21800.0    21800.0    21800.0   
      15fw   184900.0 NaN   184900.0   184900.0   184900.0   184900.0   
      15ss   208136.0 NaN   208136.0   208136.0   208136.0   208136.0   
00002 14fw    46450.0 NaN    46450.0    46450.0    46450.0    46450.0   
...               ...  ..        ...        ...        ...        ...   
19381 15fw  3143508.0 NaN  3143508.0  3143508.0  3143508.0  3143508.0   
      15ss  2471071.0 NaN  2471071.0  2471071.0  2471071.0  2471071.0   
19382 15fw  2567044.0 NaN  2567044.0  2567044.0  2567044.0  2567044.0   
      15ss  1651639.0 NaN  1651639.0  1651639.0  1651639.0  1651639.0   
19383 15fw    48600.0 NaN    48600.0    48600.0    48600.0    48600.0   

                       
                  max  
고객번호  반기               
00001 14fw   376140.0  
      14ss    21800.0  
      15fw   184900.0  
      15ss   208136.0  
00002 14fw    46450.0  
...               ...  
19381 15fw  3143508.0  
      15ss  2471071.0  
19382 15fw  2567044.0  
      15ss  1651639.0  
19383 15fw    48600.0  

[62747 rows x 16 columns]

In [92]:
pdf1 = pd.pivot_table(dfhalf,
                     index = '고객번호', # 행 위치에 들어갈 열
                     columns = '반기', # 열 위치에 들어갈 열
                     values = ['구매횟수','총구매액'], # 데이터로 사용할 열
                     aggfunc = 'max') # 데이터 집계 함수
pdf1

구매횟수                          총구매액                               
반기     14fw  14ss   15fw   15ss      14fw     14ss       15fw       15ss
고객번호                                                                    
00001  41.0   1.0   10.0   12.0  376140.0  21800.0   184900.0   208136.0
00002   7.0   1.0    NaN    8.0   46450.0  30500.0        NaN   132500.0
00003   2.0   NaN    NaN    NaN    6960.0      NaN        NaN        NaN
00004  42.0  15.0   33.0   36.0  420740.0  80060.0   759402.0   333864.0
00005   NaN   1.0    NaN    NaN       NaN  16000.0        NaN        NaN
...     ...   ...    ...    ...       ...      ...        ...        ...
19379   NaN   NaN   10.0   19.0       NaN      NaN    97690.0   203290.0
19380   NaN   NaN    5.0    2.0       NaN      NaN    30000.0    30280.0
19381   NaN   NaN  424.0  367.0       NaN      NaN  3143508.0  2471071.0
19382   NaN   NaN  505.0  394.0       NaN      NaN  2567044.0  1651639.0
19383   NaN   NaN   16.0    NaN       NaN      NaN    48600.0        NaN

[18575 rows x 8 columns]

In [95]:
lst=['14ss','14fw','15ss','15fw']
for i in lst:
    pdf1[f'반기별평균금액{i}']=round(pdf1['총구매액'][i]/pdf1['구매횟수'][i])
pdf1

구매횟수                          총구매액                                 \
반기     14fw  14ss   15fw   15ss      14fw     14ss       15fw       15ss   
고객번호                                                                       
00001  41.0   1.0   10.0   12.0  376140.0  21800.0   184900.0   208136.0   
00002   7.0   1.0    0.0    8.0   46450.0  30500.0        0.0   132500.0   
00003   2.0   0.0    0.0    0.0    6960.0      0.0        0.0        0.0   
00004  42.0  15.0   33.0   36.0  420740.0  80060.0   759402.0   333864.0   
00005   0.0   1.0    0.0    0.0       0.0  16000.0        0.0        0.0   
...     ...   ...    ...    ...       ...      ...        ...        ...   
19379   0.0   0.0   10.0   19.0       0.0      0.0    97690.0   203290.0   
19380   0.0   0.0    5.0    2.0       0.0      0.0    30000.0    30280.0   
19381   0.0   0.0  424.0  367.0       0.0      0.0  3143508.0  2471071.0   
19382   0.0   0.0  505.0  394.0       0.0      0.0  2567044.0  1651639.0   
19383   0.0   0.0   16.0    0.0       0.0      0.0    48600.0        0.0   

        반기별평균금액14ss   반기별평균금액14fw   반기별평균금액15ss   반기별평균금액15fw  
반기                                                             
고객번호                                                           
00001  21800.000000   9174.146341  17344.666667  18490.000000  
00002  30500.000000   6635.714286  16562.500000           NaN  
00003           NaN   3480.000000           NaN           NaN  
00004   5337.333333  10017.619048   9274.000000  23012.181818  
00005  16000.000000           NaN           NaN           NaN  
...             ...           ...           ...           ...  
19379           NaN           NaN  10699.473684   9769.000000  
19380           NaN           NaN  15140.000000   6000.000000  
19381           NaN           NaN   6733.163488   7413.933962  
19382           NaN           NaN   4191.977157   5083.255446  
19383           NaN           NaN           NaN   3037.500000  

[18575 rows x 12 columns]

In [96]:
pdf1.fillna(0,inplace=True)
pdf1

구매횟수                          총구매액                                 \
반기     14fw  14ss   15fw   15ss      14fw     14ss       15fw       15ss   
고객번호                                                                       
00001  41.0   1.0   10.0   12.0  376140.0  21800.0   184900.0   208136.0   
00002   7.0   1.0    0.0    8.0   46450.0  30500.0        0.0   132500.0   
00003   2.0   0.0    0.0    0.0    6960.0      0.0        0.0        0.0   
00004  42.0  15.0   33.0   36.0  420740.0  80060.0   759402.0   333864.0   
00005   0.0   1.0    0.0    0.0       0.0  16000.0        0.0        0.0   
...     ...   ...    ...    ...       ...      ...        ...        ...   
19379   0.0   0.0   10.0   19.0       0.0      0.0    97690.0   203290.0   
19380   0.0   0.0    5.0    2.0       0.0      0.0    30000.0    30280.0   
19381   0.0   0.0  424.0  367.0       0.0      0.0  3143508.0  2471071.0   
19382   0.0   0.0  505.0  394.0       0.0      0.0  2567044.0  1651639.0   
19383   0.0   0.0   16.0    0.0       0.0      0.0    48600.0        0.0   

        반기별평균금액14ss   반기별평균금액14fw   반기별평균금액15ss   반기별평균금액15fw  
반기                                                             
고객번호                                                           
00001  21800.000000   9174.146341  17344.666667  18490.000000  
00002  30500.000000   6635.714286  16562.500000      0.000000  
00003      0.000000   3480.000000      0.000000      0.000000  
00004   5337.333333  10017.619048   9274.000000  23012.181818  
00005  16000.000000      0.000000      0.000000      0.000000  
...             ...           ...           ...           ...  
19379      0.000000      0.000000  10699.473684   9769.000000  
19380      0.000000      0.000000  15140.000000   6000.000000  
19381      0.000000      0.000000   6733.163488   7413.933962  
19382      0.000000      0.000000   4191.977157   5083.255446  
19383      0.000000      0.000000      0.000000   3037.500000  

[18575 rows x 12 columns]

In [97]:
pdf1['14년도별평균금액']=pdf1['반기별평균금액14ss']+pdf1['반기별평균금액14fw']
pdf1['15년도별평균금액']=pdf1['반기별평균금액15ss']+pdf1['반기별평균금액15fw']
pdf1

구매횟수                          총구매액                                 \
반기     14fw  14ss   15fw   15ss      14fw     14ss       15fw       15ss   
고객번호                                                                       
00001  41.0   1.0   10.0   12.0  376140.0  21800.0   184900.0   208136.0   
00002   7.0   1.0    0.0    8.0   46450.0  30500.0        0.0   132500.0   
00003   2.0   0.0    0.0    0.0    6960.0      0.0        0.0        0.0   
00004  42.0  15.0   33.0   36.0  420740.0  80060.0   759402.0   333864.0   
00005   0.0   1.0    0.0    0.0       0.0  16000.0        0.0        0.0   
...     ...   ...    ...    ...       ...      ...        ...        ...   
19379   0.0   0.0   10.0   19.0       0.0      0.0    97690.0   203290.0   
19380   0.0   0.0    5.0    2.0       0.0      0.0    30000.0    30280.0   
19381   0.0   0.0  424.0  367.0       0.0      0.0  3143508.0  2471071.0   
19382   0.0   0.0  505.0  394.0       0.0      0.0  2567044.0  1651639.0   
19383   0.0   0.0   16.0    0.0       0.0      0.0    48600.0        0.0   

        반기별평균금액14ss   반기별평균금액14fw   반기별평균금액15ss   반기별평균금액15fw     14년도별평균금액  \
반기                                                                            
고객번호                                                                          
00001  21800.000000   9174.146341  17344.666667  18490.000000  30974.146341   
00002  30500.000000   6635.714286  16562.500000      0.000000  37135.714286   
00003      0.000000   3480.000000      0.000000      0.000000   3480.000000   
00004   5337.333333  10017.619048   9274.000000  23012.181818  15354.952381   
00005  16000.000000      0.000000      0.000000      0.000000  16000.000000   
...             ...           ...           ...           ...           ...   
19379      0.000000      0.000000  10699.473684   9769.000000      0.000000   
19380      0.000000      0.000000  15140.000000   6000.000000      0.000000   
19381      0.000000      0.000000   6733.163488   7413.933962      0.000000   
19382      0.000000      0.000000   4191.977157   5083.255446      0.000000   
19383      0.000000      0.000000      0.000000   3037.500000      0.000000   

          15년도별평균금액  
반기                   
고객번호                 
00001  35834.666667  
00002  16562.500000  
00003      0.000000  
00004  32286.181818  
00005      0.000000  
...             ...  
19379  20468.473684  
19380  21140.000000  
19381  14147.097450  
19382   9275.232603  
19383   3037.500000  

[18575 rows x 14 columns]

In [106]:
pdf1.loc[pdf1['14년도별평균금액']<pdf1['15년도별평균금액'],'증감']='증가'
pdf1.loc[pdf1['14년도별평균금액']>pdf1['15년도별평균금액'],'증감']='감소'
pdf1

구매횟수                          총구매액                                 \
반기     14fw  14ss   15fw   15ss      14fw     14ss       15fw       15ss   
고객번호                                                                       
00001  41.0   1.0   10.0   12.0  376140.0  21800.0   184900.0   208136.0   
00002   7.0   1.0    0.0    8.0   46450.0  30500.0        0.0   132500.0   
00003   2.0   0.0    0.0    0.0    6960.0      0.0        0.0        0.0   
00004  42.0  15.0   33.0   36.0  420740.0  80060.0   759402.0   333864.0   
00005   0.0   1.0    0.0    0.0       0.0  16000.0        0.0        0.0   
...     ...   ...    ...    ...       ...      ...        ...        ...   
19379   0.0   0.0   10.0   19.0       0.0      0.0    97690.0   203290.0   
19380   0.0   0.0    5.0    2.0       0.0      0.0    30000.0    30280.0   
19381   0.0   0.0  424.0  367.0       0.0      0.0  3143508.0  2471071.0   
19382   0.0   0.0  505.0  394.0       0.0      0.0  2567044.0  1651639.0   
19383   0.0   0.0   16.0    0.0       0.0      0.0    48600.0        0.0   

        반기별평균금액14ss   반기별평균금액14fw   반기별평균금액15ss   반기별평균금액15fw     14년도별평균금액  \
반기                                                                            
고객번호                                                                          
00001  21800.000000   9174.146341  17344.666667  18490.000000  30974.146341   
00002  30500.000000   6635.714286  16562.500000      0.000000  37135.714286   
00003      0.000000   3480.000000      0.000000      0.000000   3480.000000   
00004   5337.333333  10017.619048   9274.000000  23012.181818  15354.952381   
00005  16000.000000      0.000000      0.000000      0.000000  16000.000000   
...             ...           ...           ...           ...           ...   
19379      0.000000      0.000000  10699.473684   9769.000000      0.000000   
19380      0.000000      0.000000  15140.000000   6000.000000      0.000000   
19381      0.000000      0.000000   6733.163488   7413.933962      0.000000   
19382      0.000000      0.000000   4191.977157   5083.255446      0.000000   
19383      0.000000      0.000000      0.000000   3037.500000      0.000000   

          15년도별평균금액  증감  
반기                       
고객번호                     
00001  35834.666667  증가  
00002  16562.500000  감소  
00003      0.000000  감소  
00004  32286.181818  증가  
00005      0.000000  감소  
...             ...  ..  
19379  20468.473684  증가  
19380  21140.000000  증가  
19381  14147.097450  증가  
19382   9275.232603  증가  
19383   3037.500000  증가  

[18575 rows x 15 columns]

In [105]:
round(pdf1['증감'].value_counts()/pdf1['증감'].value_counts().sum() *100,2)

감소    51.87
증가    48.13
Name: 증감, dtype: float64

In [107]:
%%time
cursor.execute('SELECT 고객번호,소분류코드,count(구매일자) 구매횟수 ,sum(구매금액) 총구매액 FROM onlyb group by 소분류코드,고객번호 order by 소분류코드')
x = cursor.fetchall()


dfz = pd.DataFrame(x, columns=['고객번호','분류코드','구매횟수','총구매액'])
dfz

Wall time: 13.1 s


,고객번호,분류코드,구매횟수,총구매액
0,00001,1,49,490376
1,00002,1,7,51500
2,00003,1,2,6960
3,00004,1,106,704966
4,00005,1,1,16000
...,...,...,...,...
51119,19376,4,8,36300
51120,19377,4,9,51180
51121,19379,4,6,57300
51122,19381,4,34,155300


In [114]:
grp=dfz.groupby(['고객번호','분류코드'])
grp.describe()

구매횟수                                                총구매액  \
           count   mean std    min    25%    50%    75%    max count   
고객번호  분류코드                                                             
00001 1      1.0   49.0 NaN   49.0   49.0   49.0   49.0   49.0   1.0   
      4      1.0   15.0 NaN   15.0   15.0   15.0   15.0   15.0   1.0   
00002 1      1.0    7.0 NaN    7.0    7.0    7.0    7.0    7.0   1.0   
      2      1.0    9.0 NaN    9.0    9.0    9.0    9.0    9.0   1.0   
00003 1      1.0    2.0 NaN    2.0    2.0    2.0    2.0    2.0   1.0   
...          ...    ...  ..    ...    ...    ...    ...    ...   ...   
19381 4      1.0   34.0 NaN   34.0   34.0   34.0   34.0   34.0   1.0   
19382 1      1.0  844.0 NaN  844.0  844.0  844.0  844.0  844.0   1.0   
      2      1.0   24.0 NaN   24.0   24.0   24.0   24.0   24.0   1.0   
      4      1.0   31.0 NaN   31.0   31.0   31.0   31.0   31.0   1.0   
19383 1      1.0   16.0 NaN   16.0   16.0   16.0   16.0   16.0   1.0   

                                                                       \
                 mean std        min        25%        50%        75%   
고객번호  분류코드                                                              
00001 1      490376.0 NaN   490376.0   490376.0   490376.0   490376.0   
      4      300600.0 NaN   300600.0   300600.0   300600.0   300600.0   
00002 1       51500.0 NaN    51500.0    51500.0    51500.0    51500.0   
      2      157950.0 NaN   157950.0   157950.0   157950.0   157950.0   
00003 1        6960.0 NaN     6960.0     6960.0     6960.0     6960.0   
...               ...  ..        ...        ...        ...        ...   
19381 4      155300.0 NaN   155300.0   155300.0   155300.0   155300.0   
19382 1     3746153.0 NaN  3746153.0  3746153.0  3746153.0  3746153.0   
      2      342900.0 NaN   342900.0   342900.0   342900.0   342900.0   
      4      129630.0 NaN   129630.0   129630.0   129630.0   129630.0   
19383 1       48600.0 NaN    48600.0    48600.0    48600.0    48600.0   

                       
                  max  
고객번호  분류코드             
00001 1      490376.0  
      4      300600.0  
00002 1       51500.0  
      2      157950.0  
00003 1        6960.0  
...               ...  
19381 4      155300.0  
19382 1     3746153.0  
      2      342900.0  
      4      129630.0  
19383 1       48600.0  

[51124 rows x 16 columns]

In [115]:
pdf2 = pd.pivot_table(dfz,
                     index = '고객번호', # 행 위치에 들어갈 열
                     columns = '분류코드', # 열 위치에 들어갈 열
                     values = ['구매횟수','총구매액'], # 데이터로 사용할 열
                     aggfunc = 'max') # 데이터 집계 함수
pdf2

구매횟수                        총구매액                              
분류코드       1     2    3     4          1         2         3         4
고객번호                                                                  
00001   49.0   NaN  NaN  15.0   490376.0       NaN       NaN  300600.0
00002    7.0   9.0  NaN   NaN    51500.0  157950.0       NaN       NaN
00003    2.0   NaN  NaN   NaN     6960.0       NaN       NaN       NaN
00004  106.0  17.0  1.0   2.0   704966.0  710600.0  170000.0    8500.0
00005    1.0   NaN  NaN   NaN    16000.0       NaN       NaN       NaN
...      ...   ...  ...   ...        ...       ...       ...       ...
19379   21.0   2.0  NaN   6.0   182880.0   60800.0       NaN   57300.0
19380    6.0   1.0  NaN   NaN    31280.0   29000.0       NaN       NaN
19381  723.0  34.0  NaN  34.0  4598879.0  860400.0       NaN  155300.0
19382  844.0  24.0  NaN  31.0  3746153.0  342900.0       NaN  129630.0
19383   16.0   NaN  NaN   NaN    48600.0       NaN       NaN       NaN

[18575 rows x 8 columns]

In [118]:
lst=['1','2','3','4']
for i in lst:
    pdf2[f'{i}분류별평균금액']=pdf2['총구매액'][i]/pdf2['구매횟수'][i]
pdf2

구매횟수                        총구매액                                \
분류코드       1     2    3     4          1         2         3         4   
고객번호                                                                     
00001   49.0   NaN  NaN  15.0   490376.0       NaN       NaN  300600.0   
00002    7.0   9.0  NaN   NaN    51500.0  157950.0       NaN       NaN   
00003    2.0   NaN  NaN   NaN     6960.0       NaN       NaN       NaN   
00004  106.0  17.0  1.0   2.0   704966.0  710600.0  170000.0    8500.0   
00005    1.0   NaN  NaN   NaN    16000.0       NaN       NaN       NaN   
...      ...   ...  ...   ...        ...       ...       ...       ...   
19379   21.0   2.0  NaN   6.0   182880.0   60800.0       NaN   57300.0   
19380    6.0   1.0  NaN   NaN    31280.0   29000.0       NaN       NaN   
19381  723.0  34.0  NaN  34.0  4598879.0  860400.0       NaN  155300.0   
19382  844.0  24.0  NaN  31.0  3746153.0  342900.0       NaN  129630.0   
19383   16.0   NaN  NaN   NaN    48600.0       NaN       NaN       NaN   

           분류별평균금액1      분류별평균금액2  분류별평균금액3      분류별평균금액4  
분류코드                                                       
고객번호                                                       
00001  10007.673469           NaN       NaN  20040.000000  
00002   7357.142857  17550.000000       NaN           NaN  
00003   3480.000000           NaN       NaN           NaN  
00004   6650.622642  41800.000000  170000.0   4250.000000  
00005  16000.000000           NaN       NaN           NaN  
...             ...           ...       ...           ...  
19379   8708.571429  30400.000000       NaN   9550.000000  
19380   5213.333333  29000.000000       NaN           NaN  
19381   6360.828492  25305.882353       NaN   4567.647059  
19382   4438.569905  14287.500000       NaN   4181.612903  
19383   3037.500000           NaN       NaN           NaN  

[18575 rows x 12 columns]

In [120]:
pdf2.fillna(0,inplace=True)
pdf2

구매횟수                        총구매액                                \
분류코드       1     2    3     4          1         2         3         4   
고객번호                                                                     
00001   49.0   0.0  0.0  15.0   490376.0       0.0       0.0  300600.0   
00002    7.0   9.0  0.0   0.0    51500.0  157950.0       0.0       0.0   
00003    2.0   0.0  0.0   0.0     6960.0       0.0       0.0       0.0   
00004  106.0  17.0  1.0   2.0   704966.0  710600.0  170000.0    8500.0   
00005    1.0   0.0  0.0   0.0    16000.0       0.0       0.0       0.0   
...      ...   ...  ...   ...        ...       ...       ...       ...   
19379   21.0   2.0  0.0   6.0   182880.0   60800.0       0.0   57300.0   
19380    6.0   1.0  0.0   0.0    31280.0   29000.0       0.0       0.0   
19381  723.0  34.0  0.0  34.0  4598879.0  860400.0       0.0  155300.0   
19382  844.0  24.0  0.0  31.0  3746153.0  342900.0       0.0  129630.0   
19383   16.0   0.0  0.0   0.0    48600.0       0.0       0.0       0.0   

           분류별평균금액1      분류별평균금액2  분류별평균금액3      분류별평균금액4  
분류코드                                                       
고객번호                                                       
00001  10007.673469      0.000000       0.0  20040.000000  
00002   7357.142857  17550.000000       0.0      0.000000  
00003   3480.000000      0.000000       0.0      0.000000  
00004   6650.622642  41800.000000  170000.0   4250.000000  
00005  16000.000000      0.000000       0.0      0.000000  
...             ...           ...       ...           ...  
19379   8708.571429  30400.000000       0.0   9550.000000  
19380   5213.333333  29000.000000       0.0      0.000000  
19381   6360.828492  25305.882353       0.0   4567.647059  
19382   4438.569905  14287.500000       0.0   4181.612903  
19383   3037.500000      0.000000       0.0      0.000000  

[18575 rows x 12 columns]

In [ ]:
#신규고객을 배제하고 기존고객의 구매패턴 분류별 사용금액을 퍼센테이지로 나눠 분석